**44. Si decimos que la ubicación de un usuario es el promedio de la latitud y longitud de
los contenidos geolocalizados para los cuales editó la última versión (ignorar
usuarios que no editaron contenido geolocalizado). ¿Cuáles son los dos usuarios
más cercanos? (⭐⭐⭐)**

In [ ]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Para este ejercicio voy a utilizar el csv **contents** y **geo_tags**

In [ ]:
downloaded = drive.CreateFile({'id':'1SnCVFJvTsaEPrmh-PpVGH6-toBVsD6NL'})
downloaded.GetContentFile('contents.csv')

downloaded2 = drive.CreateFile({'id':'1qLu79VkVJxWp9i4Pnq9cxSLjW6Nipxp9'})
downloaded2.GetContentFile('geo_tags.csv')

Como contents.csv es muy grande solo voy a leer algunas columnas para ahorrar memoria

In [ ]:
contents = pd.read_csv('contents.csv', encoding='latin-1' , usecols=['id', 'revision_id', 'revision_timestamp', 'revisor_username', 'revisor_id'], \
                            dtype={'title':'category','revisor_username':'category'})

contents.head()

,id,revision_id,revision_timestamp,revisor_username,revisor_id
0,5,132533307,2021-01-19T18:40:34Z,MetrÃ³nomo,1708233.0
1,7,138035057,2021-08-31T18:27:01Z,NaN,NaN
2,10,137966826,2021-08-28T17:17:54Z,Traitrich,4642283.0
3,15,137986788,2021-08-29T17:16:00Z,Lojwe,5338508.0
4,17,130869554,2020-11-12T19:56:15Z,Jialxv,3615366.0


Hay varias columnas de geo_tags que no voy a necesitar, así que algunas directamente no las leo.

In [ ]:
geo_tags = pd.read_csv('geo_tags.csv', encoding='latin-1', usecols=["gt_page_id", "gt_globe", "gt_lat", "gt_lon"],\
                            dtype={'gt_globe':'category'})
geo_tags.head()

,gt_page_id,gt_globe,gt_lat,gt_lon
0,4328020,earth,41.5075,-5.8175
1,4670424,earth,13.7380,-89.2920
2,4670424,earth,13.7020,-89.2080
3,4670424,earth,13.4000,-89.7500
4,4670424,earth,12.8500,-88.0000


Como solo me va a interesar el contenido geolocalizado, hago un merge por id entre estos csv:

In [ ]:
contents_geo = contents.merge(geo_tags, how='inner', left_on='id', right_on='gt_page_id')
contents_geo.head()

,id,revision_id,revision_timestamp,revisor_username,revisor_id,gt_page_id,gt_globe,gt_lat,gt_lon
0,7,138035057,2021-08-31T18:27:01Z,NaN,NaN,7,earth,42.500000,1.500000
1,10,137966826,2021-08-28T17:17:54Z,Traitrich,4642283.0,10,earth,-34.599722,-58.381944
2,15,137986788,2021-08-29T17:16:00Z,Lojwe,5338508.0,15,earth,42.506111,1.522222
3,26,133531913,2021-02-25T18:09:01Z,InternetArchiveBot,4704851.0,26,earth,43.162222,-2.070000
4,39,138047554,2021-09-01T05:01:16Z,Traitrich,4642283.0,39,earth,34.532778,69.165833


In [ ]:
contents_geo['id']

0                7
1               10
2               15
3               26
4               39
            ...   
537300    10010550
537301    10010559
537302    10010567
537303    10010581
537304    10010600
Name: id, Length: 537305, dtype: int64

Podemos ver que del contenido que teniamos en contents.csv hay **537305** geolocalizados

Calculo la ubicación de cada usuario, haciendo el promedio entre las latitudes y longitudes de los contenidos de los cuales fue su ultimo editor.

(revisor_username: Username del autor de la última revisión)

In [ ]:
grouped = contents_geo.groupby(['revisor_username']).agg({'gt_lat':['mean'], 'gt_lon':['mean'] })
grouped.columns = grouped.columns.get_level_values(0) + '_' + grouped.columns.get_level_values(1)
grouped = grouped.dropna()
grouped.reset_index(inplace=True)
grouped = grouped.rename(columns = {'gt_lat_mean':'lat','gt_lon_mean':'lon'})
grouped

,revisor_username,lat,lon
0,"""BF CLUB""",46.733439,22.023909
1,&beer&love,30.788290,-0.431666
2,(SVPG),-12.067886,-77.038516
3,-jem-,37.138333,7.382305
4,-sasha-,38.786371,-9.267020
...,...,...,...
7431,Patagonia44LS,-40.794895,-73.212875
7432,StephiLara,-31.910542,-71.132547
7433,Xavier Laumain,39.476856,-0.388431
7434,ÐÐ»ÐµÐºÑÐ°Ð½Ð´Ñ ÐÐ¾ÑÐ¸Ð½,55.751489,37.548586


Voy a utilizar un BallTree para poder buscar el vecino más cercano de cada usuario. (Autorizado por Nati)

In [ ]:
from sklearn.neighbors import BallTree

query_lats = grouped['lat']
query_lons = grouped['lon']

bt = BallTree(np.deg2rad(grouped[['lat', 'lon']].values), metric='haversine')
distances, indices = bt.query(np.deg2rad(np.c_[query_lats, query_lons]), k=2)

El array distances tendrá las distancias de los 2 vecinos más cercanos, donde siempre el primer valor será 0 (distancia a el mismo) y el segundo tendrá un valor distinto de cero.

El array indices tendrá también 2 valores, el primero será el mismo y el 2do será el indice del vecino más cercano (distinto de él)


Aunque en este DataFrame hay casos particulares, los cuales son usuarios en exactamente la misma ubicación (esto hace que a veces se calcule primero la distancia al otro y luego a si mismo (los datos estarían cruzados))


Paso las distancias a KM multiplicando por 6371 que es el radio de la tierra en km.

In [ ]:
radio = 6371
for i in range(0, len(distances)):
  distances[i][1] = distances[i][1] * radio

print(distances)

[[     0.         364950.68316668]
 [     0.         683497.09501665]
 [     0.           3363.12077247]
 ...
 [     0.           2228.96557749]
 [     0.          19190.6820882 ]
 [     0.          10321.89807672]]


In [ ]:
print(indices)

[[   0 5840]
 [   1  539]
 [   2  765]
 ...
 [7433 7309]
 [7434  132]
 [7435 7047]]


Vamos a dividir estos datos en columnas individuales y las cargamos al dataframe.

In [ ]:
d_1, d_2 = np.split(distances, 2, axis=1)
i_1, i_2 = np.split(indices, 2, axis=1)
grouped['i_1'] = i_1
grouped['d_1'] = d_1
grouped['i_2'] = i_2
grouped['d_2'] = d_2
grouped.head()

,revisor_username,lat,lon,i_1,d_1,i_2,d_2
0,"""BF CLUB""",46.733439,22.023909,0,0.0,5840,364950.683167
1,&beer&love,30.788290,-0.431666,1,0.0,539,683497.095017
2,(SVPG),-12.067886,-77.038516,2,0.0,765,3363.120772
3,-jem-,37.138333,7.382305,3,0.0,2994,641342.936076
4,-sasha-,38.786371,-9.267020,4,0.0,6200,62164.861255


Busco la distancia minima:

In [ ]:
grouped['d_1'].min()

0.0

In [ ]:
grouped['d_2'].min()

0.0

In [ ]:
grouped[(grouped['d_1'] == 0) & (grouped['d_2'] == 0)]

,revisor_username,lat,lon,i_1,d_1,i_2,d_2
533,DobleTmatutino,-33.350000,-70.516667,2347,0.0,533,0.0
640,Erichmoron,10.450000,-64.166667,7013,0.0,640,0.0
1018,Javihabbo,43.318333,11.331389,2401,0.0,1018,0.0
1663,Periodista Deportivo 2023,22.767000,108.388000,1663,0.0,4747,0.0
2347,Ankoe28,-33.350000,-70.516667,2347,0.0,533,0.0
2401,Brucointestino,43.318333,11.331389,2401,0.0,1018,0.0
3528,Saulfabre,35.689722,139.692222,6607,0.0,3528,0.0
3867,Pablo Martillana,-34.601806,-58.399444,5602,0.0,3867,0.0
3953,Variann Wryn,3.521642,-76.417036,6532,0.0,3953,0.0
4196,Organigrama de Alicante 1966,42.141667,-7.976111,4196,0.0,6749,0.0


Como podemos ver arriba hay varios usuarios con distancia 0, esto es porque su ubicación es exactamente la misma.

Uno de los pares más cercanos:
**DobleTmatutino (533) y Ankoe28(2347)**

Descartemos ahora los casos de distancia 0 y veamos cuales son los más cercanos:

(la columna d_1 siempre tendrá valores 0 porqué es la distancia a si mismo o hacia otro usuario en la misma posición)

In [ ]:
grouped['d_1'].sum()

0.0

In [ ]:
grouped = grouped.loc[grouped['d_2'] > 0]

Filtro por el nuevo minimo:

In [ ]:
grouped[grouped['d_2'] == (grouped['d_2'].min())]

,revisor_username,lat,lon,i_1,d_1,i_2,d_2
1441,Mesterderebeldia,19.435,-99.131389,1441,0.0,1619,0.073486
1619,Padaguan,19.435,-99.131389,1619,0.0,1441,0.073486


Estos 2 usuarios son los más cercanos si no tenemos en cuenta los que están ubicados en la misma posición.

Se encuentran a 0.000012 km (1,2cm).

Una forma para evitar los casos de usuarios en misma posición podría ser eliminar los duplicados con lo siguiente:

In [ ]:
grouped.drop_duplicates(['lat', 'lon'], keep='first')